### Objetivo: Construir um Classificador de Churn
- Não existe coluna a Churn ainda
- Definição do problema de negócios
- Definição do Cliente e do Não Cliente

In [1]:
import pandas as pd
import numpy as np

from scipy.stats import ttest_ind, shapiro, levene, mannwhitneyu

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [2]:
%reload_ext watermark
%watermark -a "Modelo Classificador - Churn"
%watermark --iversions

Author: Modelo Classificador - Churn

scipy : 1.9.3
pandas: 2.2.3
numpy : 1.25.2



In [3]:
df_1 = pd.read_csv('datasets/parte1.csv')
df_2 = pd.read_csv('datasets/parte2.csv')
df_3 = pd.read_csv('datasets/parte3.csv')
df_4 = pd.read_csv('datasets/parte4.csv')


df_1 = df_1.dropna()

df_2 = df_2.dropna()

df_3.TEMPO_PERMANENCIA_CLIENTE_MESES = df_3.TEMPO_PERMANENCIA_CLIENTE_MESES.astype(int)
df_3 = df_3[['ID','TEMPO_PERMANENCIA_CLIENTE_MESES']]


df_4 = df_4.dropna()
df_4["DATA_MENSAL"] = pd.to_datetime(df_4["DATA_MENSAL"])

## RFV
- Recência: mede quantos dias se passaram desde a última atividade
- Frequência: quantos meses o cliente esteve ativo
- Valor: total de casos do cliente

In [4]:
df_rfv = df_4.copy()

max_date = df_rfv["DATA_MENSAL"].max()
df_rfv['DATA_MENSAL'] = df_rfv['DATA_MENSAL'].dt.to_period('M').dt.to_timestamp()

recency = (max_date - df_rfv.groupby('ID')["DATA_MENSAL"].max()).dt.days

frequency = df_rfv.groupby("ID")["DATA_MENSAL"].nunique()

value = df_rfv.groupby("ID")["TOTAL_CASOS"].sum()

df_rfv = pd.DataFrame({
    'ID': recency.index,
    'RECENCIA': recency.values,
    'FREQUENCIA': frequency.values,
    'VALOR': value.values
})

In [5]:
df_merged = pd.merge(df_1, df_2, how='left', on='NUMERO_DO_CASO')
df_merged = pd.merge(df_merged, df_3, how='left', left_on='ID', right_on='ID')
df_merged = pd.merge(df_merged, df_4, how='left', left_on='ID', right_on='ID')

df_merged.drop(columns=['TOTAL_FATURADO_DOLAR',
                        'DATA_ABERTURA',
                        'TOTAL_CASOS_y'],inplace=True)

df_merged.rename(columns={'TOTAL_CASOS_x':'TOTAL_CASOS'},inplace=True)

df_merged.head(1)

,NUMERO_DO_CASO,TOTAL_CASOS,TOTAL_NAO_ABANDONADOS,TOTAL_ABANDONADOS,SERVICO_AGRUPADO,DATA_ABERTURA_CASO,TIPO_PAGADOR,ID,TEMPO_PERMANENCIA_CLIENTE_MESES,DATA_MENSAL,TOTAL_DOLAR_FATURADO
0,46027,1,0,1,A;B;C,01/2000,Escritório,642.00,0.00,2000-01-01,845.00


In [6]:
display(df_merged.isna().sum())
print(df_merged.shape)

NUMERO_DO_CASO                       0
TOTAL_CASOS                          0
TOTAL_NAO_ABANDONADOS                0
TOTAL_ABANDONADOS                    0
SERVICO_AGRUPADO                     0
DATA_ABERTURA_CASO                 246
TIPO_PAGADOR                       246
ID                                 246
TEMPO_PERMANENCIA_CLIENTE_MESES    246
DATA_MENSAL                        248
TOTAL_DOLAR_FATURADO               248
dtype: int64

(2180336, 11)


In [7]:
df_faturamento = df_4.groupby('ID')['TOTAL_DOLAR_FATURADO'].sum().reset_index()

In [8]:
df_ticket_medio = df_4.groupby('ID')['TOTAL_DOLAR_FATURADO'].mean().reset_index()
df_ticket_medio.rename(columns={'TOTAL_DOLAR_FATURADO':'TICKET_MEDIO'},inplace=True)

In [9]:
df_servicos = df_merged[['ID', 'SERVICO_AGRUPADO']]

df_exploded = (
    df_servicos.assign(SERVICO_AGRUPADO=df_servicos['SERVICO_AGRUPADO'].str.split(';'))
    .explode('SERVICO_AGRUPADO')
)

servico_counts = df_exploded['SERVICO_AGRUPADO'].value_counts()
corte = 500000
frequentes = servico_counts[servico_counts >= corte]
raros = servico_counts[servico_counts < corte]
substituicoes_outros = {letra: letra for letra in frequentes.index}
for letra in raros.index:
    substituicoes_outros[letra] = 'OUTROS'

df_exploded['SERVICO_AGRUPADO'] = df_exploded['SERVICO_AGRUPADO'].map(substituicoes_outros)

In [10]:
df_exploded.SERVICO_AGRUPADO.value_counts()

SERVICO_AGRUPADO
B         1378619
J         1333507
G         1293315
C         1161338
E          809786
D          718354
H          557123
OUTROS     538853
Name: count, dtype: int64

In [11]:
df_result_servicos = (
    df_exploded.groupby(['ID', 'SERVICO_AGRUPADO'])
    .size()
    .unstack(fill_value=0)
)

df_result_servicos.reset_index(inplace=True)
df_result_servicos.head()

SERVICO_AGRUPADO,ID,B,C,D,E,G,H,J,OUTROS
0,1.00,30,30,6,6,30,30,36,0
1,2.00,1,0,0,1,0,0,0,0
2,3.00,98530,78991,2672,38076,118403,108383,137274,19873
3,4.00,0,2,0,0,2,0,2,0
4,5.00,36,12,0,24,24,0,12,12


In [12]:
df_merged_abandonados = df_merged[['ID',
                                   'TOTAL_CASOS',
                                   'TOTAL_NAO_ABANDONADOS',
                                   'TOTAL_ABANDONADOS']] 

df_resultado_abandonados = df_merged_abandonados.groupby("ID").agg({
    "TOTAL_CASOS": "sum",
    "TOTAL_NAO_ABANDONADOS": "sum",
    "TOTAL_ABANDONADOS": "sum"
}).reset_index()

In [13]:
df_merged = df_merged[['ID','TEMPO_PERMANENCIA_CLIENTE_MESES']]

agg_functions = {
    'TEMPO_PERMANENCIA_CLIENTE_MESES': 'first'
}

df_merged = df_merged.groupby(['ID']).agg(agg_functions, dropna=True).reset_index()


df_merged.head()

,ID,TEMPO_PERMANENCIA_CLIENTE_MESES
0,1.00,111.00
1,2.00,0.00
2,3.00,333.00
3,4.00,0.00
4,5.00,185.00


In [14]:
df_merged_final = pd.merge(df_merged, df_result_servicos, how='left', on='ID')
df_merged_final = pd.merge(df_merged_final, df_resultado_abandonados, how='left', on='ID')
df_merged_final = pd.merge(df_merged_final, df_faturamento, how='left', on='ID')
df_merged_final = pd.merge(df_merged_final, df_ticket_medio, how='left', on='ID')
df_merged_final = pd.merge(df_merged_final, df_rfv, how='left', on='ID')


df_merged_final["PERCENTUAL_ABANDONADOS"] = (df_merged_final["TOTAL_ABANDONADOS"] / df_merged_final["TOTAL_CASOS"]) * 100

df_merged_final["PERCENTUAL_ABANDONADOS"] = df_merged_final["PERCENTUAL_ABANDONADOS"].round(2)

display(df_merged_final.head())
print(df_merged_final.shape)

,ID,TEMPO_PERMANENCIA_CLIENTE_MESES,B,C,D,E,G,H,J,OUTROS,TOTAL_CASOS,TOTAL_NAO_ABANDONADOS,TOTAL_ABANDONADOS,TOTAL_DOLAR_FATURADO,TICKET_MEDIO,RECENCIA,FREQUENCIA,VALOR,PERCENTUAL_ABANDONADOS
0,1.00,111.00,30,30,6,6,30,30,36,0,36,18,18,46430.26,7738.38,1096.00,6.00,8.00,50.00
1,2.00,0.00,1,0,0,1,0,0,0,0,1,1,0,2900.00,2900.00,6636.00,1.00,1.00,0.00
2,3.00,333.00,98530,78991,2672,38076,118403,108383,137274,19873,183199,28056,155143,3074863.95,18412.36,183.00,167.00,1115.00,84.69
3,4.00,0.00,0,2,0,0,2,0,2,0,2,0,2,4475.50,4475.50,2467.00,1.00,2.00,100.00
4,5.00,185.00,36,12,0,24,24,0,12,12,60,12,48,20573.26,1714.44,1218.00,12.00,12.00,80.00


(1596, 19)


## Testes Estatísticos

### Welch Test
- H0: não há diferença significativa nas médias das métricas entre os clientes com o percentual que foi definido como corte

- H1: há diferença significativa nas médias das métricas entre os clientes com o percentual que foi definido como corte

In [15]:
colunas_metricas = [
    'RECENCIA', 'FREQUENCIA', 'VALOR', 'TICKET_MEDIO', 'TOTAL_DOLAR_FATURADO',
    'B', 'C', 'D','E', 'G', 'H','J', 'OUTROS',
    'TEMPO_PERMANENCIA_CLIENTE_MESES','TOTAL_CASOS'
]

percentual = 60

In [16]:
grupo_churn = df_merged_final[df_merged_final['PERCENTUAL_ABANDONADOS'] >= percentual]
grupo_nao_churn = df_merged_final[df_merged_final['PERCENTUAL_ABANDONADOS'] < percentual]

resultados = []

for coluna in colunas_metricas:
    x = grupo_churn[coluna].dropna()
    y = grupo_nao_churn[coluna].dropna()
    
    if len(x) >= 3 and len(y) >= 3:
        _, p_val = ttest_ind(x, y, equal_var=False)
        resultados.append({
            'METRICA': coluna,
            'PVAL': round(p_val, 4)
        })

df_pvals_welch = pd.DataFrame(resultados)
display(df_pvals_welch)

,METRICA,PVAL
0,RECENCIA,0.00
1,FREQUENCIA,0.00
2,VALOR,0.00
3,TICKET_MEDIO,0.00
4,TOTAL_DOLAR_FATURADO,0.00
5,B,0.06
6,C,0.10
7,D,0.31
8,E,0.03
9,G,0.18


### Shapiro-Wilk
- H0: A distribuição das métricas é normal.
- H1: A distribuição das métricas não é normal.

In [17]:
grupo_churn = df_merged_final[df_merged_final['PERCENTUAL_ABANDONADOS'] >= percentual]
grupo_nao_churn = df_merged_final[df_merged_final['PERCENTUAL_ABANDONADOS'] < percentual]

resultados = []

for coluna in colunas_metricas:
    x = grupo_churn[coluna].dropna()
    y = grupo_nao_churn[coluna].dropna()
    
    if 3 <= len(x) <= 5000 and 3 <= len(y) <= 5000:
        _, p_churn = shapiro(x)
        _, p_nao_churn = shapiro(y)
        resultados.append({
            'METRICA': coluna,
            'PVAL_CHURN': round(p_churn, 4),
            'PVAL_NAO_CHURN': round(p_nao_churn, 4)
        })

df_shapiro = pd.DataFrame(resultados)
display(df_shapiro)

,METRICA,PVAL_CHURN,PVAL_NAO_CHURN
0,RECENCIA,0.00,0.00
1,FREQUENCIA,0.00,0.00
2,VALOR,0.00,0.00
3,TICKET_MEDIO,0.00,0.00
4,TOTAL_DOLAR_FATURADO,0.00,0.00
5,B,0.00,0.00
6,C,0.00,0.00
7,D,0.00,0.00
8,E,0.00,0.00
9,G,0.00,0.00


#### Interpretação do Teste: Nenhuma métrica segue uma distribuição normal

### Levene Test
- H0: As variâncias entre os dois grupos são iguais.
- H1: As variâncias entre os dois grupos são diferentes.

Verifica a homocedasticidade (homogeneidade das variâncias)

In [18]:
grupo_churn = df_merged_final[df_merged_final['PERCENTUAL_ABANDONADOS'] >= percentual]
grupo_nao_churn = df_merged_final[df_merged_final['PERCENTUAL_ABANDONADOS'] < percentual]

resultados = []

for coluna in colunas_metricas:
    x = grupo_churn[coluna].dropna()
    y = grupo_nao_churn[coluna].dropna()
    
    if len(x) >= 3 and len(y) >= 3:
        stat, p_val = levene(x, y)
        resultados.append({
            'METRICA': coluna,
            'PVAL_LEVENE': round(p_val, 4)
        })

df_levene = pd.DataFrame(resultados)
display(df_levene)

,METRICA,PVAL_LEVENE
0,RECENCIA,0.64
1,FREQUENCIA,0.00
2,VALOR,0.01
3,TICKET_MEDIO,0.00
4,TOTAL_DOLAR_FATURADO,0.00
5,B,0.06
6,C,0.10
7,D,0.32
8,E,0.04
9,G,0.18


#### Interpretação do Teste: Essas métricas têm variâncias estatisticamente diferentes entre churn ≥ 60% e < 60%.

### Mann-Whitney
- H0: As distribuições da métrica são iguais entre os grupos (clientes com ≥ 60% de abandono e clientes com < 60%).
- H1: As distribuições da métrica são diferentes entre os grupos.

In [19]:
grupo_churn = df_merged_final[df_merged_final['PERCENTUAL_ABANDONADOS'] >= percentual]
grupo_nao_churn = df_merged_final[df_merged_final['PERCENTUAL_ABANDONADOS'] < percentual]

resultados = []

for coluna in colunas_metricas:
    x = grupo_churn[coluna].dropna()
    y = grupo_nao_churn[coluna].dropna()
    
    if len(x) >= 3 and len(y) >= 3:
        stat, p_val = mannwhitneyu(x, y, alternative='two-sided')
        resultados.append({
            'METRICA': coluna,
            'PVAL_MANN_WHITNEY': round(p_val, 4)
        })

df_mannwhitney = pd.DataFrame(resultados)
display(df_mannwhitney)

,METRICA,PVAL_MANN_WHITNEY
0,RECENCIA,0.00
1,FREQUENCIA,0.00
2,VALOR,0.00
3,TICKET_MEDIO,0.00
4,TOTAL_DOLAR_FATURADO,0.00
5,B,0.00
6,C,0.00
7,D,0.00
8,E,0.00
9,G,0.00


### Interpretação dos testes em geral
Posso usar a regra dos 60%, estatisticamente é bem sustentada

In [20]:
df_merged_final["ABANDONADO"] = df_merged_final["PERCENTUAL_ABANDONADOS"].apply(lambda x: "SIM" if x >= percentual else "NAO")

df_merged_final.ABANDONADO.value_counts(normalize=True)

ABANDONADO
NAO   0.52
SIM   0.48
Name: proportion, dtype: float64

In [21]:
df_merged_final.rename(columns={'ABANDONADO': 'CHURN'},inplace=True)

df_merged_final['CHURN'].value_counts(normalize=True)

CHURN
NAO   0.52
SIM   0.48
Name: proportion, dtype: float64

In [22]:
df_merged_final.drop(columns=['TOTAL_NAO_ABANDONADOS',
                              'TOTAL_ABANDONADOS',
                              'PERCENTUAL_ABANDONADOS',
                              'TOTAL_CASOS'], inplace=True)

In [23]:
df_merged_final.CHURN.replace({'SIM':1, 'NAO':0},inplace=True)
df_merged_final.to_csv('datasets/df_ml.csv', index=False)